In [1]:
%load_ext Cython

In [2]:
import time

import cython
import numpy as np
import pandas as pd
from scipy.stats import ks_2samp

# inputs
n_cols = 50
n_rows = 1000

ks_mode = 'exact'

In [3]:
# create data
df = pd.DataFrame(
    np.random.rand(n_rows, n_cols),
    columns=[f'col{col}' for col in range(n_cols)]
)

# randomly select a base and focus window
df['window'] = np.where(np.random.rand(n_rows) >= 0.5, 'base', 'focus')

# print some info
print(df.shape)
print(df.head())
print(df['window'].value_counts())

(1000, 51)
       col0      col1      col2      col3      col4      col5      col6  \
0  0.124420  0.724192  0.845418  0.010481  0.856895  0.848444  0.899300   
1  0.859060  0.816350  0.351214  0.933498  0.762073  0.779568  0.392752   
2  0.835503  0.000217  0.926777  0.409010  0.268180  0.189532  0.323780   
3  0.876337  0.559503  0.905445  0.090122  0.224518  0.828467  0.763861   
4  0.278137  0.537268  0.828098  0.728186  0.494793  0.624010  0.595739   

       col7      col8      col9  ...     col41     col42     col43     col44  \
0  0.843155  0.827073  0.064264  ...  0.469534  0.858486  0.821279  0.907004   
1  0.714354  0.535237  0.240830  ...  0.758717  0.975035  0.899563  0.166719   
2  0.807131  0.678111  0.885234  ...  0.732950  0.904996  0.436684  0.340177   
3  0.202247  0.900465  0.558277  ...  0.583093  0.966136  0.210382  0.037612   
4  0.286915  0.697087  0.625112  ...  0.695132  0.821744  0.482091  0.164079   

      col45     col46     col47     col48     col49  wind

In [4]:
# create two numpy array's from df
arr_base = df[df['window'] == 'base']._get_numeric_data().values
arr_focus = df[df['window'] == 'focus']._get_numeric_data().values
print(type(arr_base), arr_base.shape)
print(type(arr_focus), arr_focus.shape)

<class 'numpy.ndarray'> (509, 50)
<class 'numpy.ndarray'> (491, 50)


In [5]:
def ks_df_dumb(df, ks_mode):
    """Take in a df, loop over each column, split into base and focus, and apply test.
    """
    results = []
    for col in df._get_numeric_data():
        base = df[df['window'] == 'base'][col].values
        focus = df[df['window'] == 'focus'][col].values
        ks_stat, p_value = ks_2samp(base, focus, mode=ks_mode)
        results.append((ks_stat, p_value))
    return results

In [6]:
%%timeit -n 5 -r 5
results = ks_df_dumb(df, ks_mode)

296 ms ± 15.6 ms per loop (mean ± std. dev. of 5 runs, 5 loops each)


In [7]:
print('ks_df_dumb')
start_time = time.time()
results = ks_df_dumb(df, ks_mode)
end_time = time.time()
print(f'{round(end_time-start_time,2)} seconds')

ks_df_dumb
0.33 seconds


In [8]:
def ks_df_vec(df, ks_mode):
    """Take in a df, and use np.vectorize to avoid pandas loop.
    """
    
    def my_ks_2samp(a,b):
        return ks_2samp(a,b,mode='asymp')
    
    results = []
    base = df[df['window'] == 'base']._get_numeric_data().transpose().values
    focus = df[df['window'] == 'focus']._get_numeric_data().transpose().values
    ks_2samp_vec = np.vectorize(ks_2samp, signature='(n),(m)->(),()')
    results = ks_2samp_vec(base, focus)
    results = list(zip(results[0], results[1]))
    return results

In [9]:
%%timeit -n 5 -r 5
results = ks_df_vec(df, ks_mode)

241 ms ± 63.4 ms per loop (mean ± std. dev. of 5 runs, 5 loops each)


In [10]:
print('ks_df_vec')
start_time = time.time()
results = ks_df_vec(df, ks_mode)
end_time = time.time()
print(f'{round(end_time-start_time,2)} seconds')

ks_df_vec
0.21 seconds


In [11]:
def ks_np_dumb(arr_a, arr_b, ks_mode):
    results = []
    for n in range(arr_a.shape[1]):        
        ks_stat, p_value = ks_2samp(arr_a[:,n],arr_b[:,n], mode=ks_mode)
        results.append((ks_stat, p_value))
    return results

In [12]:
%%timeit -n 5 -r 5
results = ks_np_dumb(arr_base, arr_focus, ks_mode)

522 ms ± 60.8 ms per loop (mean ± std. dev. of 5 runs, 5 loops each)


In [13]:
print('ks_np_dumb')
start_time = time.time()
results = ks_np_dumb(arr_base, arr_focus, ks_mode)
end_time = time.time()
print(f'{round(end_time-start_time,2)} seconds')

ks_np_dumb
0.61 seconds


In [14]:
def ks_np_vec(arr_a, arr_b, ks_mode):
    
    def my_ks_2samp(a,b):
        return ks_2samp(a,b,mode=ks_mode)
    
    ks_2samp_vec = np.vectorize(my_ks_2samp, signature='(n),(m)->(),()')
    results = ks_2samp_vec(arr_a.T, arr_b.T)
    results = list(zip(results[0], results[1]))
    return results

In [15]:
%%timeit -n 5 -r 5
results = ks_np_vec(arr_base, arr_focus, ks_mode)

521 ms ± 31.9 ms per loop (mean ± std. dev. of 5 runs, 5 loops each)


In [16]:
print('ks_np_vec')
start_time = time.time()
results = ks_np_vec(arr_base, arr_focus, ks_mode)
end_time = time.time()
print(f'{round(end_time-start_time,2)} seconds')

ks_np_vec
0.53 seconds


In [38]:
%%cython -a

import numpy as np
cimport numpy as cnp
cimport cython
from scipy.stats import ks_2samp


cdef double[:, :] cy_ks_np(double[:, :] arr_a, double[:, :] arr_b, str ks_mode):

    cdef double k, p
    cdef Py_ssize_t i
    cdef Py_ssize_t m = arr_a.shape[1]
    
    #result = np.zeros((m, 2), dtype=np.double)
    cdef double[:, :] result
    cdef double[:, :] result_view = result

    for i in range(m):
        k, p = ks_2samp(arr_a[:,i], arr_b[:,i], mode=ks_mode)
        result_view[i,0] = k
        result_view[i,1] = p

    return result

In [36]:
%%timeit -n 5 -r 5
results = cy_ks_np(arr_base, arr_focus, ks_mode)

535 ms ± 19.2 ms per loop (mean ± std. dev. of 5 runs, 5 loops each)


In [39]:
print('cy_ks_np')
start_time = time.time()
results = cy_ks_np(arr_base, arr_focus, ks_mode)
end_time = time.time()
print(f'{round(end_time-start_time,2)} seconds')

cy_ks_np
0.5 seconds


In [20]:
%prun cy_ks_np(arr_base, arr_focus, ks_mode)

         180454 function calls in 0.574 seconds

   Ordered by: internal time

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
       50    0.388    0.008    0.551    0.011 stats.py:5187(_compute_prob_inside_method)
    25450    0.072    0.000    0.072    0.000 {method 'cumsum' of 'numpy.ndarray' objects}
    25450    0.028    0.000    0.128    0.000 fromnumeric.py:2252(cumsum)
    51050    0.021    0.000    0.021    0.000 {built-in method builtins.min}
    25600    0.021    0.000    0.106    0.000 fromnumeric.py:54(_wrapfunc)
    25500    0.013    0.000    0.013    0.000 {built-in method builtins.max}
    25600    0.007    0.000    0.007    0.000 {built-in method builtins.getattr}
      100    0.005    0.000    0.005    0.000 {method 'searchsorted' of 'numpy.ndarray' objects}
      100    0.005    0.000    0.005    0.000 {method 'sort' of 'numpy.ndarray' objects}
       50    0.004    0.000    0.572    0.011 stats.py:5385(ks_2samp)
      100    0.001    0.000  